<a href="https://colab.research.google.com/github/addaia/TechnicalProject/blob/main/new_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets
!pip install sympy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.5 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset
import re
import ast
import operator as op
import re
from sympy import Eq, solve
from sympy.parsing.sympy_parser import parse_expr, standard_transformations, implicit_multiplication_application

In [4]:
def evaluate_expression(expr):
    """
    evaluates an arithmetic expression given as a string or solves an equation if one is provided
    """


    if "=" in expr:
        # equation mode


        transformations = standard_transformations + (implicit_multiplication_application,)
        left_str, right_str = expr.split("=", 1)
        left_expr = parse_expr(left_str.strip(), transformations=transformations)
        right_expr = parse_expr(right_str.strip(), transformations=transformations)
        equation = Eq(left_expr, right_expr)

        free_symbols = list(equation.free_symbols)

        # if there is one free symbol, return its solution directly.
        if len(free_symbols) == 1:
            symbol = free_symbols[0]
            sol = solve(equation, symbol)
            return sol[0] if len(sol) == 1 else sol
        else:
            # for multiple variables, return the complete solution set.
            sol = solve(equation)
            return sol
    else:
        # arithmetic evaluation mode using AST
        operators = {
            ast.Add: op.add,
            ast.Sub: op.sub,
            ast.Mult: op.mul,
            ast.Div: op.truediv,
            ast.Pow: op.pow,
            ast.USub: op.neg,
            ast.Mod: op.mod,
        }
        def _eval(node):
            if isinstance(node, ast.Constant):
                return node.value
            elif isinstance(node, ast.Num):
                return node.n
            elif isinstance(node, ast.BinOp):
                left = _eval(node.left)
                right = _eval(node.right)
                return operators[type(node.op)](left, right)
            elif isinstance(node, ast.UnaryOp):
                return operators[type(node.op)](_eval(node.operand))
            else:
                raise TypeError(f"Unsupported expression: {node}")

        parsed_expr = ast.parse(expr, mode='eval').body
        return _eval(parsed_expr)

In [5]:
# load the GSM8K dataset
gsm8k = load_dataset("openai/gsm8k", "main")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md:   0%|          | 0.00/7.94k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [6]:
def add_code_column(example):
    answer = example["answer"]
    # extract all substrings between << and >>
    matches = re.findall(r"<<(.*?)>>", answer)
    if not matches:
        example["code"] = ""
        return example

    # process each consecutive pair of matches
    for i in range(len(matches) - 1):
        # split the previous match into its left-hand side (calculation) and right-hand side (result).
        parts_prev = matches[i].split("=")
        if len(parts_prev) != 2:
            continue  # skip to avoid errors
        prev_calc = parts_prev[0].strip()
        prev_result = parts_prev[1].strip()

        # split the next match
        parts_curr = matches[i+1].split("=")
        if len(parts_curr) != 2:
            continue
        curr_calc = parts_curr[0].strip()
        curr_result = parts_curr[1].strip()

        # wrap in parenthesis
        replacement = f"({prev_calc})"

        # replace
        new_curr_calc = re.sub(r'\b' + re.escape(prev_result) + r'\b', replacement, curr_calc)

        # reassamble
        matches[i+1] = new_curr_calc + "=" + curr_result

    # get string
    calc_part = matches[-1].strip().split('=')[0].strip()
    example["code"] = calc_part
    return example

gsm8k = gsm8k.map(add_code_column)


Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

Map:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [7]:
gsm8k['train'][342]

{'question': 'In a 90-minute soccer game, Mark played 20 minutes, then rested after. He then played for another 35 minutes. How long was he on the sideline?',
 'answer': 'Mark played for a total of 20 + 35 = <<20+35=55>>55 minutes.\nSo, he was on the sideline for 90 - 55 = <<90-55=35>>35 minutes.\n#### 35',
 'code': '90-(20+35)'}

SOME ERROS ARE STILL


In [ ]:
wrong_count = 0

def check_matches(example, idx):

    global wrong_count
    try:
        code_expr = example["code"]
        evaluated_value = evaluate_expression(code_expr)

        # extract answwer.
        match = re.search(r"####\s*(\S+)", example["answer"])
        if not match:
            return example

        actual_answer_str = match.group(1)
        actual_answer_str = actual_answer_str.replace(',', '') # some answer are like 88,112
        try:
            actual_value = int(actual_answer_str)
        except ValueError:
            actual_value = float(actual_answer_str)

        # evaluate
        if isinstance(evaluated_value, float) and evaluated_value.is_integer():
            evaluated_value = int(evaluated_value)

        if evaluated_value != actual_value:
            wrong_count += 1
            print(f"{idx}:")
            #print(example["question"])
            print("Code expression:", code_expr, "evaluates to", evaluated_value, "but expected", actual_value)

    except Exception as e:
        print( idx, e)
    return example

# check all entries
gsm8k = gsm8k.map(check_matches, with_indices=True)

# print number of wronmg asnwers
print("Number of wrong questions:", wrong_count)


Parameter 'function'=<function check_matches at 0x79ae8470bec0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

193:
Code expression: 192 = Y + Y * 0.2 evaluates to 160.000000000000 but expected 160
289:
Code expression: x+3+2x+12+8x=301 evaluates to 26 but expected 220
303:
Code expression: 48/12 evaluates to 4 but expected 50
343:
Code expression: (((4*125)*7)/1000)*0.14 evaluates to 0.49000000000000005 but expected 49
375:
Code expression: X + X*0.8 = 36 evaluates to 20.0000000000000 but expected 20
497:
Code expression: 150/5 evaluates to 30 but expected 10000
525 invalid syntax (<unknown>, line 1)
529:
Code expression: 50/2 evaluates to 25 but expected 26
552:
Code expression: (400*500)*.8 evaluates to 160000 but expected 16000
566:
Code expression: 8*80 evaluates to 640 but expected 1280
581 <class 'ast.UAdd'>
593:
Code expression: 34-(6+18) evaluates to 10 but expected 20
596 <class 'ast.UAdd'>
601:
Code expression: 90-(40*1.5) evaluates to 30 but expected 7
609:
Code expression: 80-42 evaluates to 38 but expected 20
618 invalid syntax (<unknown>, line 0)
623:
Code expression: 18/(1.5) ev

Map:   0%|          | 0/1319 [00:00<?, ? examples/s]

13:
Code expression: (5*2)+2 evaluates to 12 but expected 18
14:
Code expression: 16-((20-(20*20/100))*25/100) evaluates to 12 but expected 60
24 invalid syntax (<unknown>, line 0)
29:
Code expression: (33*2)+33 evaluates to 99 but expected 104
30:
Code expression: (11/(7+11)*162)+10 evaluates to 109.00000000000001 but expected 109
34:
Code expression: 5+(40/2) evaluates to 25 but expected 23
43:
Code expression: 300/5 evaluates to 60 but expected 48
81:
Code expression: 9+28 evaluates to 37 but expected 17
88 invalid syntax (<unknown>, line 0)
98:
Code expression: 30-5 evaluates to 25 but expected 5
107:
Code expression: (7-1)-1 evaluates to 5 but expected 3
114:
Code expression: 1800/6 evaluates to 300 but expected 6
135:
Code expression: 20-5 evaluates to 15 but expected 40
136 invalid syntax (<unknown>, line 0)
140:
Code expression: 20-(12+4) evaluates to 4 but expected 20
141:
Code expression: 100 evaluates to 100 but expected 400
160:
Code expression: 17+7 evaluates to 24 but exp

In [8]:
def update_code(example, idx):
    if idx == 8:
        example["code"] = "184-(30+46+38+11+18)"

    if idx == 29:
        example["code"] = "2p/5 = (p + 600)/4"

    if idx == 23:
        example["code"] = "(75-(35+(2*10)))/4"

    if idx == 32:
        example["code"] = "23-(5+8+0)"

    if idx == 44:
        example["code"] = "1000 * (100 - 30 - 15 - 25)/100"

    if idx == 48:
        example["code"] = "(24/6) + 6"

    if idx == 53:
        example["code"] = "(10 * 4) - (4*1)"

    if idx == 71:
        example["code"] = "(3*12)+7 - 18"

    if idx == 85:
        example["code"] = "(2+5) -2"

    if idx == 109: # wrong explanation in dataset
        example["code"] = "100 = A + 2*A + 2*A"

    if idx == 123:
        example["code"] = "((6/12) - (12/48))*100"

    if idx == 135:
        example["code"] = "4 * x + 600 = 10000"

    if idx == 143:
        example["code"] = "15*((100-25)/25)"

    if idx == 150:
        example["code"] = "(8m) + m = 27"

    if idx == 167: # WRONG ANSWER
        example['answer'] = "The first movie was 2*60=<<2*60=120>>120 minutes\nSo this movie is 120*.6=<<120*.6=72>>72 minutes longer\nSo this movie is 192 minutes\nIt also cost 50*2=$<<50*2=100>>100 per minute to film\nSo it cost 192*100=$1920\n#### 19200"
        example["code"] = "((60*2)*1.6)*(50*2)"

    if idx == 184:
        example["code"] = "(((2000+500)-50)+1500)-200"

    if idx == 193:
        example["code"] = "192 = Y + Y * 0.2"
    if idx == 198:
        example["code"] = "(20-(6+3+6+(6*(20/60))))* (60/6)"
    if idx == 232:
        example["code"] = "(15)-1-4-2"
    if idx == 242:
        example["code"] = "((3-2)+(2-1.5)+(4-2.5))*12"
    if idx == 256:
        example["code"] = "((3+3/8) * 8 ) / (1/4)"
    if idx == 264:
        example["code"] = "(5*600+((600*2)*2))*4"
    if idx == 289:
        example["code"] = "x+3+2x+12+8x=301"
    if idx == 293:
        example["code"] = "6*(1+3+5)"
    if idx == 302:
        example["code"] = "((1*2)*(1+3/4))*2"
    if idx == 323:
        example["code"] = "(1+1+(22+36))/5"
    if idx == 330:
        example["code"] = "220*((100-5)/100)"
    if idx == 339:
        example["code"] = "44*(1/2)*(1/2)"
    if idx == 343:
        example["code"] = "(((4*125)*7)/1000)*0.14"
    if idx == 345:
        example["code"] = "(40*((100-10)/100))*30"
    if idx == 356:
        example["answer"] = "If James only missed one question in all five rounds of five questions, he correctly answered 5 * 5 - 1 = <<5*5-1=24>>24 questions\nBefore the bonus, James earned 24 correct answers * 2 points/answer = <<24*2=48>>48 points.\nSince missing one question would only disqualify him from receiving a bonus in one round, he would earn a bonus for 5-1=<<5-1=4>>4 rounds.\nHe should expect a bonus of 4 rounds* 4 points/round= <<4*4=16>>16 points.\nIncluding his bonus, James scored a total of 48 + 16 = <<48+16=64>>64 points.\n#### 64"
        example["code"] = "((5*5)-1)*2 + (5-1)*4"
    if idx == 359:
        example["code"] = "(30+27+25)*(240/60)"
    if idx == 365:
        example["code"] = "(8*5)+((8+2)*3)"

    if idx == 375:
        example["code"] = "X + X*0.8 = 36"

    if idx == 386:
        example["code"] = "60+(100*((100-20)/100))"

    if idx == 393:
        example["code"] = "W + W + 2W + 2W = 30"

    if idx == 394:
        example["code"] = "((4*28)*5)/10"

    if idx == 415:
        example["code"] = "((6+6)*2)*2 + 30"

    if idx == 455:
        example["code"] = "((((1*2)*(1*2))*2)*2)/2 "

    if idx == 464:
        example["code"] = "C + (C/2) = 30"

    if idx == 473:
        example["code"] = "J + (J-40) = 250"

    if idx == 492:
        example["code"] = "40 * ((100- (10+60))/100)"

    if idx == 515:
        example["code"] = "((((32-8)/3)*5)/100)*((100-40)/100)*100"

    if idx == 525:
        example["code"] = "(((12*2)+12+6+8)/25) * 2.5"

    if idx == 529:
        example["code"] = "(50/2)+1"

    if idx == 552:
        example["code"] = "((400*500)*(80/100))/10"

    if idx == 566:
        example["code"] = "(8*80)*2"

    if idx == 581:
        example["code"] = "(X+10) + (X+10)*1/2 = 45"

    if idx == 593:
        example["code"] = "X - (X/2) + 6 + 18 = 34"

    if idx == 596:
        example["code"] = "(12.5+(12.5*2)+((2*(12.5))-8.5))*60"

    if idx == 601:
        example["code"] = "210/((60-40)*1.5)"

    if idx == 609:
        example["code"] = "(80-42)-18"

    if idx == 618:
        example["code"] = "(40+20) * (50/100) * (50/100)"

    if idx == 623:
        example["code"] = "(18/(1+1/4+1/4)) * (1+1/4)"

    if idx == 636:
        example["code"] = "(14*3)+10"

    if idx == 657:
        example["code"] = "900-(1*60 + 20 +10)"

    if idx == 675:
        example["code"] = "100*((70/100)*(50/100)-(30/100)*(50/100))"

    if idx == 691:
        example["code"] = "10 - (18 + 6 - 24)"

    if idx == 735:
        example["code"] = "(10/5) * 60"

    if idx == 744:
        example["code"] = "20 * (X + 15) = 40 * X"

    if idx == 753:
        example["code"] = "(29+17)*7"

    if idx == 764:
        example["code"] = "(3/4)*40 - 25"

    if idx == 786: # int
        example["code"] = "(((((3*50)*52)+1200)/2)/500)/3"

    if idx == 795:
        example["code"] = "3+(3*2)+1"

    if idx == 802:
        example["code"] = "14 * (2- (1/2))"

    if idx == 807:
        example["code"] = "7 + x = 2 * x"

    if idx == 874:
        example["code"] = "(14*25) - 50"

    if idx == 894:
        example["code"] = "35 * (2/5) * (50/100)"

    if idx == 913:
        example["code"] = "(310+(960/2))-610"

    if idx == 953:
        example["code"] = "(4*2)-1+(10-4)"

    if idx == 959:
        example["code"] = "50 / (50+150)"

    if idx == 983:
        example["code"] = "((2*24)-12)/12"

    if idx == 991:
        example["code"] = "(((9*1000)/3)*4)*6"
    if idx == 1017:
        example["code"] = "3*(20*(1/2))+2"
    if idx == 1028:
        example["code"] = "20*(90+(90+20)+(90+20)/2)"
    if idx == 1072:
        example["code"] = "(200*3)+(200*5)"
    if idx == 1078:
        example["code"] = "31*2+3"
    if idx == 1090:
        example["code"] = "150+30-15"
    if idx == 1094:
        example["code"] = "2/(((16-4)*4)+2)*100"
    if idx == 1116:
        example["code"] = "(500-10)/70"
    if idx == 1135:
        example["code"] = "48-(5*2)"
    if idx == 1137:
        example["code"] = "X + (X-7)/4 = 52"
    if idx == 1147:
        example["code"] = "80*(9000-6400)"
    if idx == 1154:
        example["code"] = "45*(20-20*0.1+5)"
    if idx == 1234:
        example["code"] = "80 * (95/100) * (75/100)"
    if idx == 1242:
        example["code"] = "(300-20-10-5-2.5)-250.5"
    if idx == 1253:
        example["code"] = "12+4+12+(2*12)+1+20+10"
    if idx == 1270:
        example["code"] = "(280-40)/(10*12)"
    if idx == 1287:
        example["code"] = "3*(8/2)"
    if idx == 1323:
        example["code"] = "(6*24)**(1/2)"
    if idx == 1400:
        example["code"] = "(2*6)+10"
    if idx == 1403:
        example["code"] = "1*10000+1*1000+0*100+2*10+6"
    if idx == 1409:
        example["code"] = "(2*10)/5"
    if idx == 1420:
        example["code"] = "3w - 6 + w = 54"
    if idx == 1421:
        example["code"] = "84-(21+28)"
    if idx == 1427:
        example["code"] = "200+100-40-60"
    if idx == 1435:
        example["code"] = "10*0.25+3*0.1+3*0.05+5*0.01"
    if idx == 1451:
        example["code"] = "100000+365*1000-20000"
    if idx == 1460:
        example["code"] = "(24-4*4)/4"

    if idx == 1476:
        example["code"] = "(34+(34/((100-15)/100))+((34/((100-15)/100)) + 5))-100"
    if idx == 1479:
        example["code"] = "(60-20)/10"


    if idx == 1534:
        example["code"] = "(104*100)/(100-35)"

    if idx == 1552:
        example["code"] = "2x+20 = 180"


    if idx == 1567:
        example["code"] = "7-(3+3)"
    if idx == 1575:
        example["code"] = "200*10*20-2000"
    if idx == 1586:
        example["code"] = "280/(2+20/60)"
    if idx == 1599:
        example["code"] = "26-(9+4+5)"
    if idx == 1607:
        example["code"] = "m+(1/3)*m=76"
    if idx == 1608:
        example["code"] = "12/3-3"
    if idx == 1617:
        example["code"] = "(0.15*2+0.25+0.35)*100"
    if idx == 1624:
        example["code"] = "6+3*(6-2)"
    if idx == 1628:
        example["code"] = "(1/2)+(1/3)+(1/6)"
    if idx == 1636:
        example["code"] = "((7.5-1.5)/5)*60"
    if idx == 1661:
        example["code"] = "2025-20-3-2"
    if idx == 1673:
        example["code"] = "3.75+2.4+11.85-10"

    if idx == 1680:
        example["code"] = "349-108-153"
    if idx == 1707:
        example["code"] = "K + K +30 = 160"

    if idx == 1722:
        example["code"] = "6/((3/150)+(2/(150+50))+(1/(150*2)))"

    if idx == 1727:
        example["code"] = "4 *  (2*(25/100)) + (9-4) * 0.2"

    if idx == 1756: # int
        example["code"] = "(1200 + 400 + 200 + 60 + 200 )/15"

    if idx == 1776: # wrong answer
        example["answer"] = "Let x be the score that William needs to achieve.\nSince there are 30 people in the class, there are 30 - 1 = <<30-1=29>>29 people without him.\nSince their average was a 74%, this means that (29 * 74 + x) / 30 >= 75.\nThus, 29 * 74 + x >= 30 * 75\nThus, 29 * 74 + x >= 2250\nThus, 2146 + x >= 2250\nThus, x >= 2250 - 2146\nThus, x >= 104\n#### 104"
        example["code"] = "74*(30-1) + X = 75*30 "

    if idx == 1786:
        example["code"] = "(((4/5)*(1/4))+((1/2)*(1/5)))*100"

    if idx == 1793:
        example["code"] = "3600/(2*12)"
    if idx == 1814:
        example["code"] = "(5/(6+9+5))*100"
    if idx == 1817:
        example["code"] = "(2*500+(500-0.4*500)+(500-0.7*500))/50"

    if idx == 1841:
        example["code"] = "1+(20*(20*(5/100)/100))"
    if idx == 1842:
        example["code"] = "((50-25-50*0.2)/(50+40))*100"
    if idx == 1852:
        example["code"] = "(100-20)/2"
    if idx == 1870:
        example["code"] = "2*((180-60)/3)"
    if idx == 1885:
        example["code"] = "(2*(4*400)-400)/4"
    if idx == 1901:
        example["code"] = "1+(150/50)+1"
    if idx == 1938:
        example["code"] = "(30-(2*3+6*3))/3"

    if idx == 1941:
        example["code"] = "4*(84-30)"
    if idx == 1942:
        example["code"] = "3*10-(3*10*0.9)"
    if idx == 1949:
        example["code"] = "1.5+4.35+12.65-6.5"
    if idx == 1971:
        example["code"] = "(0.15*273600)/10"
    if idx == 1983:
        example["code"] = "((3)+(2*3))/(12+2.25+3.75+3+6)*100"

    if idx == 1990:
        example["code"] = "1970+10+20"

    if idx == 1997:
        example["code"] = "4000*(2/100)-20"
    if idx == 2011:
        example["code"] = "(2*4*60)/30"
    if idx == 2020:
        example["code"] = "(10/5)*120"
    if idx == 2025:
        example["code"] = "2*12*5"
    if idx == 2028:
        example["code"] = "((200+200/2)-(3/4*(200+200/2))+300)-150"
    if idx == 2032:
        example["code"] = "250-120+35-15"
    if idx == 2033:
        example["code"] = "425/5-23"

    if idx == 2059:
        example["code"] = "(1/4) = y / (y+11+7)"

    if idx == 2076:
        example["code"] = "(140000*(10/100))*3"

    if idx == 2078:
        example["code"] = "40*0.5*0.3"
    if idx == 2084:
        example["code"] = "2*4+4+3*2+3+1"

    if idx == 2106:
        example["code"] = "7.5 * 12 = x + 2*x"

    if idx == 2133:
        example["code"] = " X + (X - 127) = 827"

    if idx == 2146:
        example["code"] = "14*3-10"
    if idx == 2154:
        example["code"] = "(300*4)/3-100"
    if idx == 2168:
        example["code"] = "10000 * ((100-40)/100) * ((100-30)/100)"
    if idx == 2182:
        example["code"] = "8*4+12*4"
    if idx == 2204:
        example["code"] = "((150-135)/150)*100"
    if idx == 2218:
        example["code"] = "((8-6)/8)*100"
    if idx == 2241:
        example["code"] = "15 * (X+1) = 30 * X"
    if idx == 2244:
        example["code"] = "9*8-5"
    if idx == 2250:
        example["code"] = "20*6/16+0.5"
    if idx == 2265:
        example["code"] = "((50+30)*2)-130"
    if idx == 2270:
        example["code"] = "(12/(25-10))*100"

    if idx == 2287:
        example["code"] = "312 = p + 6 + p + 4 * (6+p)"
    if idx == 2293:
        example["code"] = "1.5*(60/45)"
    if idx == 2299:
        example["code"] = "(12/(12+18))*100"
    if idx == 2304:
        example["code"] = "100-3*1"
    if idx == 2369:
        example["code"] = "500+50*(20-14)"

    if idx == 2394:
        example["code"] = "84 = r+ 6 *r"
    if idx == 2405:
        example["code"] = "12+((255-150)/15)"
    if idx == 2416:
        example["code"] = "((6-3)/(6+2*6+(6-3)))*100"

    if idx == 2429:
        example["code"] = "25 * 10 - 28 *8"
    if idx == 2439:
        example["code"] = "2*16+7"
    if idx == 2444:
        example["code"] = "750-(10*25+7*50+1*100)"
    if idx == 2445:
        example["code"] = "7*(3*3)+9-3"

    if idx == 2452:
        example["code"] = "50 = 10 + 2* x"

    if idx == 2456:
        example["code"] = "72 + 74 + 3x = 437"

    if idx == 2472:
        example["code"] = "(3/5)*(1/5)*100"
    if idx == 2473:
        example["code"] = "2*(12/3)+5"
    if idx == 2494:
        example["code"] = "905-(723+32+123)"
    if idx == 2512:
        example["code"] = "(5*2+4*2+4*3.5+3*1+(2*2.5))/4"
    if idx == 2516:
        example["code"] = "((2+2)/((2+2)+(4+2)))*100"

    if idx == 2517:
        example["code"] = "80000*(100+20+5)/100"
    if idx == 2525:
        example["code"] = "2*(14+5)-6"
    if idx == 2541:
        example["code"] = "x = 30 * (100/200)"

    if idx == 2562:
        example["code"] = "3 * (60/20) * x = 27"

    if idx == 2575:
        example["code"] = "(40*0.7)/(35/5)"
    if idx == 2581:
        example["code"] = "50-(2*2+2*1)"

    if idx == 2582:
        example["code"] = "x + 2*x + 2*2*x = 140"
    if idx == 2584:
        example["code"] = "((200-(42+67+51))/200)*100"



    if idx == 2600:
        example["code"] = "5+11+5-19"
    if idx == 2612:
        example["code"] = "((210/7)*15)/45*21"
    if idx == 2656:
        example["code"] = "200+200/2+(200+200/2)+400"

    if idx == 2680:
        example["code"] = "(2+(6/12)) + 3 * (2+(6/12)) "
    if idx == 2686:
        example["code"] = "x+ (1/2)*x + (1/2)*x+1 =25"
    if idx == 2720:
        example["code"] = "33-(12+5+4)"
    if idx == 2725:
        example["code"] = "(3/4*(40*0.6))*10-130"
    if idx == 2770:
        example["code"] = "5*10+10*2+6*3+10*2-2"
    if idx == 2781:
        example["code"] = "(60 +12)/2"
    if idx == 2802:
        example["code"] = "(4*15+8*10)*0.8"

    if idx == 2822:
        example["code"] = "J + 10 = 3 * 30"
    if idx == 2849:
        example["code"] = "(4.65+1.15)-(0.95+0.85)"
    if idx == 2850:
        example["code"] = "2000-2000*0.05"
    if idx == 2877: # ><
        example["code"] = "1500/75"
    if idx == 2885:
        example["code"] = "(27+14+21)-(5+17)"

    if idx == 2899:
        example["code"] = "X - 1/2*X - 14 - 1/3*X - 16 = 0"
    if idx == 2906:
        example["code"] = "20*2+10*4+5*2"
    if idx == 2917:
        example["code"] = "120*2/3"

    if idx == 2921:
        example["code"] = "b + b * (1/3) + 4= 52"
    if idx == 2935:
        example["code"] = "17.80-(4*0.95+6)"
    if idx == 2936:
        example["code"] = "516-(516/3+516/2)"
    if idx == 2963:
        example["code"] = "((29+3)*2)-4"
    if idx == 2969:
        example["code"] = "(2*20)/(80*2)*100"
    if idx == 3014:
        example["code"] = "((638-27)*3)+100-27*4"
    if idx == 3022:
        example["code"] = "500000 = (x*1.25)* (100/(100-75))"

    if idx == 3026:
        example["code"] = "(8*12)/(6*9+8*12)*100"
    if idx == 3036:
        example["code"] = "((2/5*10)*2.5)+((3/5*10)*2)"

    if idx == 3055:
        example["code"] = "(100-20) * (100-20) /100"
    if idx == 3057:
        example["code"] = "(6-5)+3"
    if idx == 3062:
        example["code"] = "8*(1+2.5)/2"
    if idx == 3065:
        example["code"] = "((80*2)*2)*1.1"
    if idx == 3089:
        example["code"] = "99 = x + x*10"
    if idx == 3116:
        example["code"] = "((124-58+24)-47+14)+10"

    if idx == 3148:
        example["code"] = "(100 - X/5) + X = 244"
    if idx == 3153:
        example["code"] = "6/(6+6-2)*100"

    if idx == 3157:
        example["code"] = "(65-45)/0.25"
    if idx == 3210:
        example["code"] = "5p + p = 24"
    if idx == 3221:
        example["code"] = "(32/2-5)*4"
    if idx == 3232:
        example["code"] = "100000*1.1*0.9"
    if idx == 3234:
        example["code"] = "(124600/2)/10"
    if idx == 3263:
        example["answer"] = 'First find how many candy canes Andy gets from his teachers: 3 canes/teacher * 4 teachers = <<3*4=12>>12 canes\nThen add the number of candy canes he gets from his parents: 12 canes + 2 canes = <<12+2=14>>14 canes\nThen divide that number by 7 to find the number of canes he buys: 14 canes / 7 = <<14/7=2>>2 canes\nThen add the number of candy canes he gets from each source to find the total number: 2 canes + 14 canes = <<2+14=16>>16 canes\nThen divide the total number of candy canes by the number of candy canes it takes to get one cavity to find the number of cavities Andy gets: 16 canes / 4 canes/cavity = <<16/4=4>>4 cavities\n#### 4'
        example["code"] = "((2+3*4)+((2+3*4)/7))/4"
    if idx == 3279:
        example["code"] = "6/(3/4)*20"
    if idx == 3298:
        example["code"] = "((200*0.63)+(100-43))/300*100"
    if idx == 3331:
        example["code"] = "(38 - x) * 10 = 20*16"
    if idx == 3332:
        example["code"] = "3x + x = 32"
    if idx == 3334:
        example["code"] = "360 * ( 5/(5+7))*(1/2)*40 + 360 * ( 7/(5+7))*(2/3)*30"

    if idx == 3339:
        example["code"] = "X+X*(20/100) = 960"

    if idx == 3346:
        example["code"] = "X - 1/5*X - 1/4*X - 120 =540"

    if idx == 3350:
        example["code"] = "48 / 2**X = 6"

    if idx == 3353:
        example["code"] = "23*15+12"

    if idx == 3361:
        example["code"] = "25+x = 2*(6+x)"

    if idx == 3362:
        example["code"] = "90 * (4/3) * 5"

    if idx == 3383:
        example["code"] = "(100-20)*20/100"

    if idx == 3390:
        example["code"] = "(10/2*5)+((9*2/3)*5)"
    if idx == 3392:
        example["code"] = "4p + p = 80"
    if idx == 3400:
        example["code"] = "8*((239+21)/13)-14"
    if idx == 3404:
        example["code"] = "300+30+25"
    if idx == 3409:
        example["code"] = "(80/8)*0.5"
    if idx == 3417:
        example["code"] = "(3+4)/20*100"

    if idx == 3423:
        example["code"] = "14*2*2-4"

    if idx == 3433:
        example["code"] = "8 * 56/60 = 7 * (56+x)/60"

    if idx == 3484:
        example["code"] = "3*4"
    if idx == 3517:
        example["code"] = "((25-5)/40)*100"
    if idx == 3520:
        example["code"] = "20/260*100"

    if idx == 3554:
        example["code"] = "5p + p = 90"

    if idx == 3558:
        example["code"] = "190 * (50/100) * (20/100)"

    if idx == 3562:
        example["code"] = "A + A/2 + (A/2)*2 + 5 = 930"
    if idx == 3565:
        example["code"] = "x+(x/4)+x+(x/4)=100"
    if idx == 3576:
        example["code"] = "(60*10-30)-(2 * 60 +  4*60 + 60)"
    if idx == 3577:
        example["code"] = "47.25 / (7 * (3/4)) - 42/(7 * (3/4))"
    if idx == 3687:
        example["code"] = "(600/10)/(600/10+60)*100"
    if idx == 3693:
        example["code"] = "40/80*100"

    if idx == 3712:
        example["code"] = "15*8/3"
    if idx == 3722:
        example["code"] = "(16+24)+5"

    if idx == 3746:
        example["code"] = "6 = p + p/2"
    if idx == 3754:
        example["code"] = "2*(180+180/6)"
    if idx == 3755:
        example["code"] = "((1077-129)/6)*1.5"
    if idx == 3807:
        example["code"] = "(24/6)*9"

    if idx == 3821:
        example["code"] = "X - X*1/4 = 24 "

    if idx == 3842:
        example["code"] = "X + 3/4*X + X+10 = 340"
    if idx == 3870:
        example["code"] = "80-(4+22)"
    if idx == 3873:
        example["code"] = "6/3"
    if idx == 3895:
        example["code"] = "(16-((16*3/8)+2))/16*100"
    if idx == 3899:
        example["code"] = "5*(56/4/2)"
    if idx == 3918:
        example["code"] = "(10+15+27+12+19)*2"

    if idx == 3923:
        example["code"] = "(80 * (25/100) * 5/4) / 1.25"
    if idx == 3928:
        example["code"] = "60*3/20"
    if idx == 3944:
        example["code"] = "2*38+6"
    if idx == 4011:
        example["code"] = "8/50*100"
    if idx == 4013:
        example["code"] = "(27+13+28)-(49+5)"
    if idx == 4045:
        example["code"] = "(2*6)/(2.5*6)*100"
    if idx == 4062:
        example["code"] = "(181+5*7)/(18*30*4)*100"
    if idx == 4079:
        example["code"] = "47-(4*2+2*2)"
    if idx == 4086:
        example["code"] = "50*3+10"

    if idx == 4102:
        example["code"] = "100/80 * 500"
    if idx == 4120:
        example["code"] = "16-((14+12+16)/3)"

    if idx == 4121:
        example["code"] = "((100-20)/100) * ((100-60)/100) * ((100-(1/4)*100)/100)*100"
    if idx == 4131:
        example["code"] = "(40000+2000+3000)/0.9"
    if idx == 4145:
        example["code"] = "400/0.2/(2/3)"

    if idx == 4159:
        example["code"] = "3c + c = 120"

    if idx == 4170:
        example["code"] = "m+(m+55) = 215"
    if idx == 4173:
        example["code"] = "2/(1/4)"
    if idx == 4176:
        example["code"] = "(24-(2*2))/4"

    if idx == 4177:
        example["code"] = "9*((439-10)/13)+10"
    if idx == 4178:
        example["code"] = "1+2+2*(100+75)/100 + 1 * (50/100)"
    if idx == 4194:
        example["code"] = "14*3/(2*7)"
    if idx == 4203:
        example["code"] = "3200-2500-350"
    if idx == 4206:
        example["code"] = "((2*60+4*(365-60))+79)/80"
    if idx == 4209:
        example["code"] = "4*(48/192)"
    if idx == 4261:
        example["code"] = "(90-5*10)/(30-10)"
    if idx == 4264:
        example["code"] = "(5*3+10*2)/70*100"
    if idx == 4267:
        example["code"] = "4000+4000*10+50000"

    if idx == 4274:
        example["code"] = "(1-2/5)*(2000-(2/5*2000))*(100-40)/100"
    if idx == 4310:
        example["code"] = "(2*70-0.1*2*70)+30"
    if idx == 4330:
        example["code"] = "(40+20)*1.2"
    if idx == 4339:
        example["code"] = "(300-255)/300*100"

    if idx == 4349:
        example["code"] = "10+2*60+41+3*60+16-6*60"
    if idx == 4358:
        example["code"] = "(5*x)/20 = 2"
    if idx == 4362:
        example["code"] = "40*0.75+50*0.98"
    if idx == 4394:
        example["code"] = "75-15-25"
    if idx == 4403:
        example["code"] = "1 + 0 + 2 + 1/2 + 2 +1/2"
    if idx == 4419:
        example["code"] = "(6*2+2*5+2*3+2*4)/10"
    if idx == 4430:
        example["code"] = "(100-20)/5"
    if idx == 4485:
        example["code"] = "2/(10/60)"
    if idx == 4504:
        example["code"] = "2*(5*8)+8"
    if idx == 4508:
        example["code"] = "(3000/2+100)+1000"
    if idx == 4534:
        example["code"] = "63/3*2"
    if idx == 4555:
        example["code"] = "(40/2)/5"
    if idx == 4593:
        example["code"] = "((366/2*0.25)+13)/14"
    if idx == 4603:
        example["code"] = "12/18*100"
    if idx == 4622:
        example["code"] = "(260-5*50)/10"
    if idx == 4644:
        example["code"] = "3*8+1"
    if idx == 4653:
        example["code"] = "4*(10+15)"
    if idx == 4677:
        example["code"] = "45-(1+10+6)"
    if idx == 4714:
        example["code"] = "(5+2+2)/27*100"
    if idx == 4752:
        example["code"] = "((8-6)/20)*100"
    if idx == 4796:
        example["answer"] = "There are 5*20 = <<5*20=100>>100 tanks in the yard.\nAltogether, there are 100+20 = <<100+20=120>>120 trucks and tanks in the yard.\n#### 120"
        example["code"] = "(5*20)+20"
    if idx == 4803:
        example["code"] = "(2*2+2+0.5+1.5)/8"
    if idx == 4814:
        example["code"] = "9*10-7"
    if idx == 4820:
        example["code"] = "(0.1+0.3+0.3)*100"
    if idx == 4846:
        example["code"] = "2**(4*60/20)"
    if idx == 4850:
        example["code"] = "((12-10)/10)*100"
    if idx == 4866:
        example["code"] = "(29+29*4)/29"
    if idx == 4873:
        example["code"] = "(0.6/(24-15))*60"

    if idx == 4881:
        example["code"] = "x+3x+(3*2)x=150"
    if idx == 4910:
        example["code"] = "10*40-5*60"
    if idx == 4920:
        example["code"] = "6d - 8 + d = 69"
    if idx == 4937:
        example["code"] = "200*60+400*30"
    if idx == 4983:
        example["code"] = "4.8*1000/(800+400)"
    if idx == 4984:
        example["code"] = " 2b - 5 = 3(b - 5)"
    if idx == 4987:
        example["code"] = "5*(20*1.25-20)"
    if idx == 4994:
        example["code"] = "x+(1/3)x+(2/3)x=132"
    if idx == 5041:
        example["code"] = "(200-((10*8)+3*5*5))/5"
    if idx == 5066:
        example["code"] = "(467000-450000)/5"
    if idx == 5127:
        example["code"] = "(16+18)/2"
    if idx == 5142:
        example["code"] = "8*8-55"
    if idx == 5145:
        example["code"] = "0.4*80+0.6*90"
    if idx == 5152:
        example["code"] = "(10+(2+1+1+0.5*4))/8"
    if idx == 5172:
        example["code"] = "8/40*100"
    if idx == 5192:
        example["code"] = "120*5+250"
    if idx == 5209:
        example["code"] = "x + (x + 5) + (x + 1) = 18"
    if idx == 5262:
        example["code"] = "30/5 - 3"
    if idx == 5270:
        example["code"] = "16*8/4"
    if idx == 5276:
        example["code"] = "8/25*100"
    if idx == 5280:
        example["code"] = "((10-2)+7)*36"
    if idx == 5282:
        example["code"] = "70/(5*7/5)"
    if idx == 5291:
        example["code"] = "(22+3)/5"
    if idx == 5307:
        example["code"] = "2*2*2"
    if idx == 5327:
        example["code"] = "2*(30+10)+5*(100*1.2)"
    if idx == 5350:
        example["code"] = "13+(13-9)+(4*(13-9))"
    if idx == 5352:
        example["code"] = "X + (X*2) + (X*2 + 1) = 31"
    if idx == 5363:
        example["code"] = "(3000/500)*2"
    if idx == 5391:
        example["code"] = "(100-90)/10"
    if idx == 5418:
        example["code"] = "480/2**X<50"
    if idx == 5431:
        example["code"] = "x + (x+5)/4 = 30"
    if idx == 5434:
        example["code"] = "x + (x-10)/4 = 340"
    if idx == 5441:
        example["code"] = "(1600-600-500)/2"
    if idx == 5468:
        example["code"] = "8*(200-200*2/5)"
    if idx == 5475:
        example["code"] = "10+2*10"
    if idx == 5497:
        example["code"] = "(84/2)*20/60"
    if idx == 5500:
        example["code"] = "2000*((100-15)/100)*((100-10)/100)-200"
    if idx == 5565:
        example["code"] = "(120/2)*0.9"
    if idx == 5578:
        example["code"] = "3*20/2"
    if idx == 5584:
        example["code"] = "((100-53)/100)*200"
    if idx == 5588:
        example["code"] = "40+10-(40*20/100 + 20)"
    if idx == 5597:
        example["code"] = "4+(30+30+10+20+90)/60"
    if idx == 5619:
        example["code"] = "(2*100+1*50+5*20+3*10+7*5+27*1)-300"
    if idx == 5629:
        example["code"] = "((1/3)+(1/3)-(1/6))*100"
    if idx == 5678:
        example["code"] = "(71-2-3)/1.5"
    if idx == 5724:
        example["code"] = "(120-60*(30/60))*(1.5-0.5)"
    if idx == 5728:
        example["code"] = "2*(2.4*0.75)+2.4"
    if idx == 5815:
        example["code"] = "7500/((100-25)/100)"
    if idx == 5816:
        example["code"] = "8*30+5*((8*30)/2)"
    if idx == 5829:
        example["code"] = "(100*30+100*25)/25"
    if idx == 5866:
        example["code"] = "50+100+20+10"
    if idx == 5872:
        example["code"] = "(15-5)*0.2"
    if idx == 5873:
        example["code"] = "(56+44)/4"
    if idx == 5878:
        example["code"] = "6*(1/4)+3*(1/2)+2*2"
    if idx == 5886:
        example["code"] = "X*20/100 = 168"
    if idx == 5907:
        example["code"] = "70/140*100"
    if idx == 5914:
        example["code"] = "X * (0.5 * 10 * 60/100 + 0.5 * 15*1/3) = 55"
    if idx == 5935:
        example["code"] = "30/60 + (30 + 30 * 1.2)/44"
    if idx == 5936:
        example["code"] = "9*100-400"
    if idx == 5946:
        example["code"] = "3/4*40+ (3/4)*40/2 "
    if idx == 5950:
        example["code"] = "3*(12/2)-4"
    if idx == 5974:
        example["code"] = "d + 7d - 3 = 45"
    if idx == 5983:
        example["code"] = "1+2+100-40 "
    if idx == 5991:
        example["code"] = "((125*1.25*0.8)/125)*100"
    if idx == 6013:
        example["code"] = "20*42/15"
    if idx == 6049:
        example["code"] = "(50*5+50*7)*0.9"
    if idx == 6089:
        example["code"] = "7.5+2.5+4+1"
    if idx == 6094:
        example["code"] = "(130-13)/9"
    if idx == 6117:
        example["code"] = "2*16-8"
    if idx == 6123:
        example["code"] = "(7000-2*20000*0.1)/(3*0.2)"
    if idx == 6152:
        example["code"] = "200=x+4*x"
    if idx == 6157:
        example["code"] = "100-(100/10)+(100/5)"
    if idx == 6169:
        example["code"] = "3*2**X>26.3"
    if idx == 6234:
        example["code"] = "X + X/2 + 4*X= 99"
    if idx == 6246:
        example["code"] = "((600-600*2/5)-((600-600*2/5)*3/5))/600*100"
    if idx == 6251:
        example["code"] = "((60*2)/5)/2"
    if idx == 6261:
        example["code"] = "2*p + 5*(20/2)=62"
    if idx == 6268:
        example["code"] = "7c + c = 560"
    if idx == 6272:
        example["code"] = "3/12*100"
    if idx == 6276:
        example["code"] = "(17.5-2)*4"
    if idx == 6280:
        example["code"] = "45/90*100"
    if idx == 6281:
        example["code"] = "3*x + x = 64"
    if idx == 6305:
        example["code"] = "3x + x = 40-8"
    if idx == 6308:
        example["code"] = "(13380-5400)/420"
    if idx == 6309:
        example["code"] = "3*(28-20)-3*(23-20)"
    if idx == 6321:
        example["code"] = "2*3+2*3*2+1*3*(1/3)+2*1.5"
    if idx == 6325:
        example["code"] = "50-((6*3)+(6*3))"
    if idx == 6351:
        example["code"] = "162-(12000/1000)"
    if idx == 6354:
        example["code"] = "((13+19)/2)+5"
    if idx == 6362:
        example["code"] = "X - 1/5*X - 1/4*(X-1/5*X) = 360"
    if idx == 6364:
        example["code"] = "100+50+300+150"
    if idx == 6368:
        example["code"] = "(5000/0.02+110000)/6"
    if idx == 6399:
        example["code"] = "50*500/5"
    if idx == 6402:
        example["code"] = "(30/2)*(1/3)"
    if idx == 6404:
        example["code"] = "((2*0.75+4*0.25)/(1/6))/1.5"
    if idx == 6407:
        example["code"] = "32/50*100"
    if idx == 6460:
        example["code"] = "100*(60/(60+40))"
    if idx == 6473:
        example["code"] = "(5*2+2)*3/2"
    if idx == 6492:
        example["code"] = "5*(30/3)*2"
    if idx == 6512:
        example["code"] = "50*12+80*20+70*16"
    if idx == 6514:
        example["code"] = "500*(40/100)*(70/100)"
    if idx == 6537:
        example["code"] = "(9*6000)/3600"
    if idx == 6539:
        example["code"] = "(5650000 - (1000000 + 1000000 + 500000)*2)/2"
    if idx == 6556:
        example["code"] = "100-3*20"
    if idx == 6564:
        example["code"] = "39 /( (50/100) *((100-35)/100))"
    if idx == 6587:
        example["code"] = "(20+10)/2"
    if idx == 6589:
        example["code"] = "(60*8*(1-0.75)*2)/80"
    if idx == 6605:
        example["code"] = "(1500-(50+9+121))/220"
    if idx == 6617:
        example["code"] = "(102-85)/85*100"
    if idx == 6625:
        example["code"] = "48-24*0.5"
    if idx == 6627:
        example["code"] = "3*2 + (7-2)*8*(60/100)"
    if idx == 6638:
        example["code"] = "z + 4z = 80"
    if idx == 6644:
        example["code"] = "2*37-18"
    if idx == 6665:
        example["code"] = "10*0.25+3*0.1+4*0.05+200*0.01"
    if idx == 6683:
        example["code"] = "10/20*100"
    if idx == 6718:
        example["code"] = "(1/3 + 1/4)/(1/12)"
    if idx == 6726:
        example["code"] = "24+3*X = 90"
    if idx == 6744:
        example["code"] = "(1600+400)/8"
    if idx == 6748:
        example["code"] = "(12*3+72+12)/3"
    if idx == 6760:
        example["code"] = "2*2+2*2+1.5*(2*2)"
    if idx == 6828:
        example["code"] = "38*2.5+56*3.5"
    if idx == 6830:
        example["code"] = "6/12*100"
    if idx == 6840:
        example["code"] = "80-40-8-12"
    if idx == 6844:
        example["code"] = "20*1800/5"
    if idx == 6845:
        example["code"] = "(3*1-2.7)*5/3+(10/5 - 3/2)*5"
    if idx == 6889:
        example["code"] = "(2000-600+800)-1200"
    if idx == 6894:
        example["code"] = "(125+125)/1250*100"
    if idx == 6899:
        example["code"] = "10+11*x = 131"
    if idx == 6925:
        example["code"] = "(20*120)/100*5"
    if idx == 6937:
        example["code"] = "3*1.5+0.5"
    if idx == 6951:
        example["code"] = "(5000*4)*(2+1)-5000"
    if idx == 6958:
        example["code"] = "a + 5a - 6 = 150"
    if idx == 7002:
        example["code"] = "12+6 - (2*16*(25/100))"
    if idx == 7017:
        example["code"] = "20*(60-40)"
    if idx == 7037:
        example["code"] = "10/20*100"
    if idx == 7039:
        example["code"] = "2*60/15"
    if idx == 7076:
        example["code"] = "45*3-20*2"
    if idx == 7077:
        example["code"] = "0.5+8/12-2/12+2*0.5"
    if idx == 7096:
        example["code"] = "(15*3)*(80/100)-3*2"
    if idx == 7102:
        example["code"] = "(1000/8)*(2250/10)"
    if idx == 7112:
        example["code"] = "(350-140)/350*100"
    if idx == 7115:
        example["code"] = "100-(25*40/100)"
    if idx == 7133:
        example["code"] = "(60+15)/15"
    if idx == 7139:
        example["code"] = "400/(2**X) = 25"
    if idx == 7146:
        example["code"] = "(9/21*100)+0.5"
    if idx == 7147:
        example["code"] = "20+6*x-2*x/2=40"
    if idx == 7148:
        example["code"] = "(280+105-200)+42"
    if idx == 7168:
        example["code"] = "60/4*0.2"
    if idx == 7182:
        example["answer"] = "Let x be the original number.\n2*x+5=20+x/2\n2*x-x/2=15\n4*x-x=30\n3*x=30\nx=<<10=10>>10\n#### 10"
        example["code"] = "2*x+5=20+x/2"
    if idx == 7189:
        example["code"] = "(13/(11 +1 +13))*100"
    if idx == 7206:
        example["code"] = "28*100/(100-((80+50)/2))"
    if idx == 7222:
        example["code"] = "100*(20/100)*(80/100)"
    if idx == 7235:
        example["code"] = "c + 5 = 2*(c-3)"
    if idx == 7241:
        example["code"] = "(400-120-80)*0.5*0.75"
    if idx == 7274:
        example["code"] = "30*20/100"
    if idx == 7291:
        example["code"] = "5*75/25"
    if idx == 7293:
        example["code"] = "c + 2c = 21"
    if idx == 7330:
        example["code"] = "(8*(1/2)+2)*(4/8)"
    if idx == 7332:
        example["code"] = "45000+(45000/3)*1.5"

    if idx == 7340:
       example["code"] = "6 = 2Y + Y"
    if idx == 7364:
        example["code"] = "3* X = 1.5 * 4 + 3*2"
    if idx == 7373:
        example["code"] = "(5+2*2) * (30+1*60)"
    if idx == 7380:
        example["code"] = "240*0.6*0.5"
    if idx == 7406:
        example["code"] = "13+11+13+11 - (12+15+9+10)"
    if idx == 7408:
        example["code"] = "(5+5/2+1/2)/2"
    if idx == 7430:
        example["code"] = "(50*4)-80"
    if idx == 7444:
        example["answer"] = "He played 7*2 = <<7*2=14>>14 gigs\nThe long song was 5*2=<<5*2=10>>10 minutes\nSo all the gigs were 5+5+10=<<5+5+10=20>>20 minutes\nSo he played 14*20=140 minutes\n#### 140"
        example["code"] = "(2*7/2)*(2+2)*5"

    if idx == 7467:
        example["code"] = "8*3=4*x"
    if idx == 7470:
        example["code"] = "6*(2/3)"


    return example

gsm8k["train"] = gsm8k["train"].map(update_code, with_indices=True)

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

In [1]:
def update_code_test(example, idx):


    if idx == 13:
        example["code"] = "x - (x/3) - 2 - ((x - (x/3) - 2)/2) = 5"
    if idx == 14:
        example["code"] = "(20 - (20*0.20) - ((20 - 20*0.20)*0.25))/20*100"
    if idx == 24:
        example["code"] = "X - X*(25/100) = 19.50"
    if idx == 29:
        example["code"] = "33 + 2*33 + 5"
    if idx == 34:
        example["code"] = "((40/2)+5)-2"
    if idx == 43:
        example["code"] = "(200/250)*(300/5)"
    if idx == 81:
        example["code"] = "(10+21+23) - (9+28)"
    if idx == 88:
        example["code"] = "10h + h = 88,000"
    if idx == 98:
        example["code"] = "30 - 5 - 20"
    if idx == 107:
        example["code"] = "(7 - (2+1.5+2))/0.5"
    if idx == 114:
        example["code"] = "1800/(6*50)"
    if idx == 135:
        example["code"] = "3*(20-5)-5"
    if idx == 136:
        example["code"] = "(27/3)*(2/3)"
    if idx == 140:
        example["code"] = "((20 - (12+4))/20)*100"
    if idx == 141:
        example["code"] = "700/7*4"
    if idx == 160:
        example["code"] = "(17+7)*(2/3)"
    if idx == 176:
        example["code"] = "(240-80)-60"
    if idx == 184:
        example["code"] = "(((3/6)-((3/6)*(3/6))))*100"
    if idx == 211:
        example["code"] = "(42-10)/8"
    if idx == 226:
        example["code"] = "((40-8)/(30+(20+14)+(40-8)))*100"
    if idx == 237:
        example["code"] = "(20+10)*3"
    if idx == 239:
        example["code"] = "(1000-300) - ((1000-300)*3/5)"
    if idx == 244:
        example["code"] = "27 - (24 - 5 - 10 - 2)"
    if idx == 266:
        example["code"] = "400000 - (0.03*400000 + 0.05*400000 + 250000)"
    if idx == 280:
        example["code"] = "20*0.25 + 4*0.50"
    if idx == 308:
        example["code"] = "(2*(13-11)) + (4*0.5)"
    if idx == 314:
        example["code"] = "D + 2*D + D/2 = 105"
    if idx == 341:
        example["code"] = "50+50+50+(50+10)+(50-20)+20"
    if idx == 348:
        example["code"] = "((5*220) - (10*100))/(10*100)*100"
    if idx == 350:
        example["code"] = "1 + 3 + 0.5 + 1.5 + 2"
    if idx == 356:
        example["code"] = "(28 - (6*2 + 5*1)) + 10"
    if idx == 360:
        example["code"] = "(45+5)/2"
    if idx == 387:
        example["code"] = "6*2 + 8*3"
    if idx == 393:
        example["code"] = "(90/450)*100"
    if idx == 394:
        example["code"] = "(2*(6*6)) + 20"
    if idx == 416:
        example["code"] = "180 - (304 - 180)"
    if idx == 420:
        example["code"] = "4000 + (2*4000) + 5000"
    if idx == 426:
        example["code"] = "4 + (4*1.25) + (4/2)"
    if idx == 431:
        example["code"] = "(6+11+2) - (2+16)"
    if idx == 433:
        example["code"] = "(15/5)*(1/4)*8"
    if idx == 440:
        example["code"] = "2*(9+(360/9))"
    if idx == 450:
        example["code"] = "20+1-2-12-3"
    if idx == 458:
        example["code"] = "((3+4)/(8+12))*100"
    if idx == 473:
        example["code"] = "((2*6/12)/10)*100"
    if idx == 482:
        example["code"] = "200 - (83+91)"
    if idx == 493:
        example["code"] = "((4-2)/(4*2))*100"
    if idx == 499:
        example["code"] = "200/(25-5)"
    if idx == 511:
        example["code"] = "(30*7) - 100"
    if idx == 519:
        example["code"] = "2*120+40"
    if idx == 576:
        example["code"] = "216*(12+2+4+34)"
    if idx == 593:
        example["code"] = "20<4*2**X"
    if idx == 598:
        example["code"] = "((8*25)+(6*10)+(14*5)+15)/5"
    if idx == 607:
        example["code"] = "24/(120*2)*100"
    if idx == 609:
        example["code"] = "(4*7)*(1/4)"
    if idx == 624:
        example["code"] = "8*14 - 9*3"
    if idx == 635:
        example["code"] = "10 - 60/10"
    if idx == 659:
        example["code"] = "((0.5*10)+(1/3*15))-((0.2*10)+(1/3*15))"
    if idx == 682:
        example["code"] = "(140-20)/1.5"
    if idx == 689:
        example["code"] = "10*0.40 + 10*0.30"
    if idx == 695:
        example["code"] = "50*0.5*0.2"
    if idx == 699:
        example["code"] = "(20-4)/2"
    if idx == 707:
        example["code"] = "40*(100-80)/100"
    if idx == 728:
        example["code"] = "((15+45)/150)*100"
    if idx == 756:
        example["code"] = "(33/3)*2"
    if idx == 763:
        example["code"] = "250=x+x+(1/2)*x"
    if idx == 769:
        example["code"] = "(5*15+5) + (2*15)"
    if idx == 783:
        example["code"] = "20 - (8+2*4)"
    if idx == 798:
        example["code"] = "20*((10+0.5+1)-(10+0.5))"
    if idx == 806:
        example["code"] = "((340+40+(400*0.8))/(500+100+400))*100"
    if idx == 808:
        example["code"] = "((4*1.5)+(16*2)+10)/4"
    if idx == 809:
        example["code"] = "100-150+100"
    if idx == 821:
        example["code"] = "(50+20)/2"
    if idx == 825:
        example["code"] = "((341-12)/7)*3 + 12"
    if idx == 842:
        example["code"] = "4*20 - 70"
    if idx == 855:
        example["code"] = "(87-9)/6"
    if idx == 856:
        example["code"] = "27*2/8"
    if idx == 858:
        example["code"] = "10*(7**3)"
    if idx == 866:
        example["code"] = "((85-1)/3)"
    if idx == 872:
        example["code"] = "3*(21/7) + 2*(8/4)"
    if idx == 873:
        example["code"] = "12.48 - 16*0.03"
    if idx == 881:
        example["code"] = "24 - (18-10)"
    if idx == 931:
        example["code"] = "2*20+10"
    if idx == 946:
        example["code"] = "7*70 - 40"
    if idx == 963:
        example["code"] = "4*40 - 13"
    if idx == 977:
        example["code"] = "(4*8)/8"
    if idx == 988:
        example["code"] = "(2000*0.25*12)/1000"
    if idx == 1012:
        example["code"] = "28 = X - 9*5 - 7*3"
    if idx == 1024:
        example["code"] = "(80*1 + 60*4) - 2*10"
    if idx == 1025:
        example["code"] = "100 - (30*2 + 10)"
    if idx == 1035:
        example["code"] = "((24/3)-1)*5"
    if idx == 1038:
        example["code"] = "(200000+25000*4)/(25000*3)"
    if idx == 1043:
        example["code"] = "26*5*1000"
    if idx == 1047:
        example["code"] = "((19-4)/2.5)+4"
    if idx == 1084:
        example["code"] = "(100-60)/2"
    if idx == 1110:
        example["code"] = "(9300 - (2*125*30))/30"
    if idx == 1113:
        example["code"] = "2 - 8 + 3"
    if idx == 1126:
        example["code"] = "(40000/2)/400"
    if idx == 1141:
        example["code"] = "((3*30)+(5*30))/12"
    if idx == 1152:
        example["code"] = "0.2*0.6*100"
    if idx == 1158:
        example["code"] = "2300*(100/20) - 4200"
    if idx == 1159:
        example["code"] = "((8*0.75*1/3)/((8*0.75*1/3)+(8*0.25)))*100"
    if idx == 1168:
        example["code"] = "4*(12-7)"
    if idx == 1173:
        example["code"] = "1/(12/120)"
    if idx == 1176:
        example["code"] = "(5-5*0.45)/(5*0.2)"
    if idx == 1179:
        example["code"] = "((4+(4+7))/30)*100"
    if idx == 1204:
        example["code"] = "700 - (700*0.4) - ((700-700*0.4)*0.6)"
    if idx == 1208:
        example["code"] = "72*(11/8 - 1)"
    if idx == 1220:
        example["code"] = "40*(4/5)"
    if idx == 1223:
        example["code"] = "(16*0.25) + ((36-16)*0.2)"
    if idx == 1225:
        example["code"] = "(10+2)/4"
    if idx == 1235:
        example["code"] = "(50*(24*3))/(6000)*100"
    if idx == 1243:
        example["code"] = "((18-3)/30)*100"
    if idx == 1245:
        example["code"] = "5*20 - 4"
    if idx == 1280:
        example["code"] = "(10/2)+2"
    return example
gsm8k["test"] = gsm8k["test"].map(update_code_test, with_indices=True)

NameError: name 'gsm8k' is not defined